<a href="https://colab.research.google.com/github/nubianaliatti/NubiaNaliattiTcc/blob/main/model_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd NubiaNaliattiTcc

[Errno 2] No such file or directory: 'NubiaNaliattiTcc'
/content


In [2]:
!pip install scikit-learn seaborn openmeteo_requests requests_cache retry_requests retry_requests pymysql -U pandas numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt


ImportError: cannot import name '_center' from 'numpy._core.umath' (/usr/local/lib/python3.12/dist-packages/numpy/_core/umath.py)

In [ ]:
import pandas as pd

# 🚫 Sem banco de dados — vamos ler direto do CSV salvo anteriormente
input_file = '248_pedro_duplicados.csv'   # arquivo gerado no seu script final.py

# Lendo os dados do CSV
df = pd.read_csv(input_file)

# Salvando no novo arquivo de saída
output_file = 'dados_treinamento.csv'
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"✅ Dados exportados com sucesso para o arquivo: {output_file}")

In [ ]:
df = pd.read_csv('dados_treinamento.csv')
print(df.head())

In [ ]:
df_to_prepare = df[['com.samsung.health.exercise.distance',
                    'com.samsung.health.exercise.mean_speed',
                    'com.samsung.health.exercise.duration', 'com.samsung.health.exercise.mean_heart_rate',
                    'sleep_duration',
                    #'temperature_x', 'humidity_x', 'sleep_score', 'mental_recovery','volume_7d'
                    'temperature_2m', 'relative_humidity_2m', 'sleep_score', 'mental_recovery'
                    ]]

print(df_to_prepare.head())

In [ ]:
'''df_to_prepare = df_to_prepare.replace({r'\.': ''}, regex=True)
df_to_prepare = df_to_prepare.astype(float)

print(df_to_prepare.dtypes)

non_numeric_columns = df_to_prepare.select_dtypes(exclude=['number']).columns
print("Colunas não numéricas:", non_numeric_columns)'''

import re
import pandas as pd
import numpy as np

# Função para detectar colunas que não são numéricas
def detectar_colunas_problematicas(df):
    problem = []
    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            problem.append(col)
    return problem

colunas_problematicas = detectar_colunas_problematicas(df_to_prepare)
print("⚠️ Colunas que precisam de limpeza:", colunas_problematicas)

def limpar_valor(v):
    if isinstance(v, str):
        # Extrai o primeiro número (inteiro ou decimal, incluindo negativos)
        numeros = re.findall(r"[-+]?\d*\.\d+|\d+", v)
        if numeros:
            return float(numeros[0])
        else:
            return None
    return v

# Aplica a função somente nas colunas problemáticas
'''colunas_com_texto = [
    'com.samsung.health.exercise.distance',
    'com.samsung.health.exercise.mean_speed',
    #'temperature_2m',
    #'relative_humidity_2m'
]'''

df_to_prepare[colunas_problematicas] = df_to_prepare[colunas_problematicas].applymap(limpar_valor)

# Confere se agora virou float
print(df_to_prepare.dtypes)
# Converte durações no formato "0 days 00:00:00.000..." para segundos
for col in df_to_prepare.columns:
    if df_to_prepare[col].astype(str).str.contains('days').any():
        df_to_prepare[col] = pd.to_timedelta(df_to_prepare[col], errors='coerce').dt.total_seconds()

df_to_prepare = df_to_prepare.replace({r'\.': ''}, regex=True)
df_to_prepare = df_to_prepare.astype(float)

print(df_to_prepare.dtypes)

non_numeric_columns = df_to_prepare.select_dtypes(exclude=['number']).columns
print("Colunas não numéricas:", non_numeric_columns)


In [ ]:
df_to_prepare.fillna(df_to_prepare.mean(), inplace=True)  # Substituir NaN pela média

#print(df_to_prepare.head())


In [ ]:
# Definir o índice para dividir os dados (80% para treino)
train_size = int(len(df) * 0.8)

train = df_to_prepare.iloc[:train_size]
test = df_to_prepare.iloc[train_size:]

X_train = train.drop(columns=['com.samsung.health.exercise.duration'])
y_train = train['com.samsung.health.exercise.duration']
X_test = test.drop(columns=['com.samsung.health.exercise.duration'])
y_test = test['com.samsung.health.exercise.duration']

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_test.to_csv("dados_de_teste.csv", index=False)
X_train.to_csv("df_treinamento.csv", index=False)


In [ ]:

for col in X_train.columns:
    if not pd.api.types.is_numeric_dtype(X_train[col]):
        print(f"Coluna não numérica encontrada: {col}")



In [ ]:


output_dir = "resultados_modelos"
os.makedirs(output_dir, exist_ok=True)

models = {
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Linear Regression": LinearRegression()
}

results = {}

for name, model in models.items():
    print(f"Treinando {name}...")

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    #rmse = mean_squared_error(y_test, y_pred, squared=False)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results[name] = {"MAE": mae, "RMSE": rmse, "R²": r2}
    print(f"{name} - MAE: {mae}, RMSE: {rmse}, R²: {r2}\n")

    output_file = os.path.join(output_dir, f"{name}_predicoes.csv")
    result_df = pd.DataFrame({
        "Valor Real": y_test,
        "Predição": y_pred,
        "Erro Absoluto": abs(y_test - y_pred)
    })
    result_df.to_csv(output_file, index=False)
    print(f"Resultados de {name} salvos em: {output_file}")

print("Resultados Finais:")
for name, metrics in results.items():
    print(f"{name}: {metrics}")


In [ ]:

models = {
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Linear Regression": LinearRegression()
}

results = {}

feature_importances = {}

for name, model in models.items():
    print(f"Treinando {name}...")

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    #rmse = mean_squared_error(y_test, y_pred, squared=False)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results[name] = {"MAE": mae, "RMSE": rmse, "R²": r2}
    print(f"{name} - MAE: {mae}, RMSE: {rmse}, R²: {r2}\n")

    if hasattr(model, "feature_importances_"):
        feature_importances[name] = model.feature_importances_

print("Resultados Finais:")
for name, metrics in results.items():
    print(f"{name}: {metrics}")

if feature_importances:
    print("\nImportância das Features:")
    for model_name, importances in feature_importances.items():
        feature_df = pd.DataFrame({
            "Feature": X_train.columns,
            "Importance": importances
        }).sort_values(by="Importance", ascending=False)
        print(f"\n{model_name} Feature Importances:")
        print(feature_df)


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[['com.samsung.health.exercise.duration']])
plt.title("Boxplot de Duração")
plt.show()

In [ ]:
Q1 = df['com.samsung.health.exercise.duration'].quantile(0.25)
Q3 = df['com.samsung.health.exercise.duration'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

print(f"Limite Inferior: {limite_inferior}, Limite Superior: {limite_superior}")


In [ ]:
df_sem_outliers = df_to_prepare[
    (df_to_prepare['com.samsung.health.exercise.duration'] >= limite_inferior) &
    (df_to_prepare['com.samsung.health.exercise.duration'] <= limite_superior)
]

outliers = df_to_prepare[
    (df_to_prepare['com.samsung.health.exercise.duration'] < limite_inferior) |
    (df_to_prepare['com.samsung.health.exercise.duration'] > limite_superior)
]

outliers.to_csv('outliers_removed.csv', index=False)

In [ ]:

train_size = int(len(df) * 0.8)

train = df_sem_outliers.iloc[:train_size]
test = df_sem_outliers.iloc[train_size:]

X_train = train.drop(columns=['com.samsung.health.exercise.duration'])
y_train = train['com.samsung.health.exercise.duration']
X_test = test.drop(columns=['com.samsung.health.exercise.duration'])
y_test = test['com.samsung.health.exercise.duration']

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:

for col in X_train.columns:
    if not pd.api.types.is_numeric_dtype(X_train[col]):
        print(f"Coluna não numérica encontrada: {col}")



In [ ]:
output_dir = "resultados_modelos_sem_outliers"
os.makedirs(output_dir, exist_ok=True)

for name, model in models.items():
    print(f"Treinando {name}...")

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    #rmse = mean_squared_error(y_test, y_pred, squared=False)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results[name] = {"MAE": mae, "RMSE": rmse, "R²": r2}
    print(f"{name} - MAE: {mae}, RMSE: {rmse}, R²: {r2}\n")

    output_file = os.path.join(output_dir, f"{name}_predicoes.csv")
    result_df = pd.DataFrame({
        "Valor Real": y_test,
        "Predição": y_pred,
        "Erro Absoluto": abs(y_test - y_pred)
    })
    result_df.to_csv(output_file, index=False)
    print(f"Resultados de {name} salvos em: {output_file}")

print("Resultados Finais:")
for name, metrics in results.items():
    print(f"{name}: {metrics}")

# Outro teste

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Padronizar os dados para modelos sensíveis a escala
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ajuste de modelos com hiperparâmetros melhores
models = {
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    "SVR": SVR(kernel='rbf', C=10, gamma=0.1),
    "KNN": KNeighborsRegressor(n_neighbors=5, weights='distance'),
    "Decision Tree": DecisionTreeRegressor(max_depth=10, min_samples_split=10, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42),
    "Linear Regression": LinearRegression()
}

results = {}
feature_importances = {}

for name, model in models.items():
    print(f"Treinando {name}...")

    if name in ["SVR", "KNN", "Gradient Boosting"]:
        X_train_used, X_test_used = X_train_scaled, X_test_scaled
    else:
        X_train_used, X_test_used = X_train, X_test

    model.fit(X_train_used, y_train)

    y_pred = model.predict(X_test_used)

    # Corrigir previsões irreais (se o erro for maior que 30% da média dos tempos reais é feito o ajuste)
    media_real = y_test.mean()
    limite_min = media_real * 0.7  # 70% da média
    limite_max = media_real * 1.3  # 130% da média
    y_pred = np.clip(y_pred, limite_min, limite_max)

    mae = mean_absolute_error(y_test, y_pred)
    #rmse = mean_squared_error(y_test, y_pred, squared=False)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results[name] = {"MAE": mae, "RMSE": rmse, "R²": r2}
    print(f"{name} - MAE: {mae}, RMSE: {rmse}, R²: {r2}\n")

    if hasattr(model, "feature_importances_"):
        feature_importances[name] = model.feature_importances_

print("Resultados Finais:")
for name, metrics in results.items():
    print(f"{name}: {metrics}")

if feature_importances:
    print("\nImportância das Features:")
    for model_name, importances in feature_importances.items():
        feature_df = pd.DataFrame({
            "Feature": X_train.columns,
            "Importance": importances
        }).sort_values(by="Importance", ascending=False)
        print(f"\n{model_name} Feature Importances:")
        print(feature_df)


In [ ]:
df_sem_outliers.loc[:, 'com.samsung.health.exercise.duration'] = df_sem_outliers['com.samsung.health.exercise.duration'].clip(lower=limite_inferior, upper=limite_superior)

print(df_sem_outliers.head(198))

In [ ]:
import joblib

# Diretório para salvar os modelos
model_filenames = {
    "Gradient Boosting": "gradient_boosting_sem_caloria.pkl",
    "SVR": "svr_sem_caloria.pkl",
    "KNN": "knn_sem_caloria.pkl",
    "Decision Tree": "decision_tree_sem_caloria.pkl",
    "Random Forest": "random_forest_sem_caloria.pkl",
    "Linear Regression": "linear_regression_sem_caloria.pkl"
}

for name, model in models.items():
    filename = model_filenames[name]
    joblib.dump(model, filename)
    print(f"Modelo {name} salvo como {filename}")


In [ ]:
import joblib
import pandas as pd

model_filenames = {
    "Gradient Boosting": "gradient_boosting_sem_caloria.pkl",
    "SVR": "svr_sem_caloria.pkl",
    "KNN": "knn_sem_caloria.pkl",
    "Decision Tree": "decision_tree_sem_caloria.pkl",
    "Random Forest": "random_forest_sem_caloria.pkl",
    "Linear Regression": "linear_regression_sem_caloria.pkl"
}

# Carrega os modelos
try:
    models = {name: joblib.load(filename) for name, filename in model_filenames.items()}
except FileNotFoundError as e:
    print(f"Erro ao carregar modelos: {e}")
    exit()

# Carrega os dados de teste
try:
    X_test = pd.read_csv("dados_de_teste.csv")  # Ajuste o caminho se necessário
except FileNotFoundError:
    print("Arquivo 'dados_de_teste.csv' não encontrado.")
    exit()

if X_test.empty:
    print("O arquivo 'dados_de_teste.csv' está vazio.")
    exit()

# Extrai a última linha dos dados de teste
ultima_linha = X_test.iloc[-1].to_dict()

# Função para fazer a previsão
def prever_duracao(model, distancia):
    dados = ultima_linha.copy()

    # Converte a distância para o mesmo tipo de dado presente no dataset
    try:
        distancia = type(dados['com.samsung.health.exercise.distance'])(distancia)
    except KeyError:
        print("Coluna 'com.samsung.health.exercise.distance' não encontrada no dataset.")
        return None, None

    dados['com.samsung.health.exercise.distance'] = distancia

    df = pd.DataFrame([dados])

    try:
        duracao_ms = model.predict(df)[0]  # Retorna a duração prevista em milissegundos
    except Exception as e:
        print(f"Erro ao fazer previsão com o modelo: {e}")
        return None, None

    # Converte milissegundos para minutos e segundos
    duracao_segundos = duracao_ms / 1000
    minutos = int(duracao_segundos // 60)
    segundos = int(duracao_segundos % 60)

    return minutos, segundos

distancia = input("Digite a distância em metros: ")

try:
    distancia = float(distancia)
except ValueError:
    print("Entrada inválida para distância. Digite um número válido.")
    exit()

# Roda todos os modelos e exibir previsões
print("\nResultados das previsões:")
for name, model in models.items():
    minutos, segundos = prever_duracao(model, distancia)
    if minutos is not None and segundos is not None:
        print(f"{name}: {minutos} minutos e {segundos} segundos.")

In [ ]:
correlation_matrix = df_to_prepare.corr()

# Exibir a matriz de correlação
print(correlation_matrix)

# Plotar como um heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação')
plt.show()